# Adversarial Training
A first approach to make the CNN model more robust to FGSM adversarial attacks. <br>
It consists of finetuning the pretrained model with both clean and adversarial examples. <br>

In [1]:
import os
import numpy as np
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from collections import Counter
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import utils


### Preprocessing

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset_folder = "mri_brain_tumor"

train_dataset = datasets.ImageFolder(root=f'{dataset_folder}/Training', transform=transform)
test_dataset = datasets.ImageFolder(root=f'{dataset_folder}/Testing', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=1)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=1)

In [3]:
mean, std = utils.compute_mean_std(train_loader)
print(f"Mean: {mean}")
print(f"Std: {std}")

100%|██████████| 714/714 [00:26<00:00, 26.54it/s]

Mean: tensor([0.1855, 0.1855, 0.1855])
Std: tensor([0.1813, 0.1813, 0.1813])


In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean.tolist(), std=std.tolist())
])
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=1)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True, num_workers=1)

### Model definition

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device.")

model = utils.CNN(num_classes=4).to(device)
#load weights
model.load_state_dict(torch.load('weights/cnn.pth'))
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


Using cuda device.


C:\Users\gnele\AppData\Local\Temp\ipykernel_2136\435087753.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('weights/cnn.pth'))


Total number of parameters: 51476484


## Train
Use both original and noisy (FGSM) images to train the model.


In [6]:

def adversarial_train(dataloader, model, device, num_epochs):

    model.train()

    for epoch in range(num_epochs):
        running_loss = 0.0
        batch_loss = 0.0

        print(f"Epoch [{epoch+1}/{num_epochs}]")
        for batch_idx, (images, labels) in enumerate(dataloader):

            images, labels = images.to(device), labels.to(device)
            attack_images = utils.fgsm_attack(model, device, criterion, images, labels, 0.1).to(device)
            #append the attack images to the images
            images = torch.cat((images, attack_images), 0)
            labels = torch.cat((labels, labels), 0)

            optimizer.zero_grad()
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            batch_loss += loss.item()

            #remove attack images to free memory
            del attack_images
            
            if batch_idx % 128 == 0:
                print(f"[{batch_idx}/{len(dataloader)}] Loss: {batch_loss/128:.4f}")
                batch_loss = 0.0
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")

    print("Training complete.")

In [6]:
#define a new model
robust_model = utils.CNN(num_classes=4).to(device)
robust_model.load_state_dict(torch.load('weights/cnn.pth'))
#adversarial_train(train_loader, robust_model, model, device, num_epochs=3)

C:\Users\gnele\AppData\Local\Temp\ipykernel_2136\1066980876.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  robust_model.load_state_dict(torch.load('weights/cnn.pth'))


<All keys matched successfully>

In [8]:
#torch.save(robust_model.state_dict(),'weights/robust_fgsm.pth')

## Evaluate

In [7]:
model.load_state_dict(torch.load('weights/cnn.pth')).to(device)
model.eval()
robust_model.load_state_dict(torch.load('weights/robust_fgsm.pth')).to(device)
robust_model.eval()

print("\n\n-------- ORIGINAL MODEL ---------")
utils.compare_eval_fgsm(model, test_loader, criterion, device)
print("\n\n-------- ROBUST MODEL ---------")
utils.compare_eval_fgsm(robust_model,test_loader, criterion, device)

C:\Users\gnele\AppData\Local\Temp\ipykernel_2136\1506686983.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('weights/cnn.pth'))
C:\Users



--------ORIGINAL MODEL---------
[0/32]
[16/32]
[32/32]

Original accuracy: 	0.91
Adversarial accuaracy: 	0.00


--------ROBUST MODEL---------
[0/32]
[16/32]
[32/32]

Original accuracy: 	0.95
Adversarial accuaracy: 	0.87
